In [0]:
%run "../../../config/setup"

In [0]:
-- Compute the average total_amount for discount_sensitivity calculation
CREATE OR REPLACE TEMPORARY VIEW avg_total_amount AS
SELECT AVG(total_amount) AS avg_total_amount FROM transactions_silver;

-- Perform the main query to create the preference_analysis table
CREATE OR REPLACE TABLE customers_preference_analysis_gold AS
WITH customer_preferences AS (
  SELECT
    c.customer_id,
    i.category,
    i.price,
    t.total_amount,
    MONTH(t.transaction_date) AS transaction_month
  FROM customers_silver c
  LEFT JOIN transactions_silver t ON c.customer_id = t.customer_id
  LEFT JOIN orders_silver o ON t.transaction_id = o.transaction_id
  LEFT JOIN items_dedupped i ON o.item_id = i.item_id
),
aggregated_preferences AS (
  SELECT
    customer_id,
    -- Calculate top_category using MODE (most frequent category)
    FIRST(category) AS top_category,
    -- Calculate preferred_price_range as "min_price-max_price"
    CONCAT(FLOOR(MIN(price)), '-', FLOOR(MAX(price))) AS preferred_price_range,
    -- Calculate discount_sensitivity (if customer has transactions below average)
    SUM(CASE WHEN total_amount < (SELECT avg_total_amount FROM avg_total_amount) THEN 1 ELSE 0 END) > 0 AS discount_sensitivity,
    -- Calculate seasonal_purchaser (if standard deviation of transaction months < 2)
    STDDEV(transaction_month) < 2 AS seasonal_purchaser
  FROM customer_preferences
  GROUP BY customer_id
)
SELECT * FROM aggregated_preferences;